In [35]:
import gmaps
import numpy as np
import pandas as pd
import requests
import time
from scipy.stats import linregress
from matplotlib import pyplot as plt

In [36]:
file = "HH_Provider_July2021.csv"

df = pd.read_csv(file)

In [37]:
df_new = df[["State","CMS Certification Number (CCN)", "Provider Name", "Address","City", "ZIP", "Type of Ownership"]]
df_new = df_new.replace("VOLUNTARY NON-PROFIT - RELIGIOUS AFFILIATION", "VOLUNTARY NON-PROFIT - RELIGIOUS")
df_new = df_new.replace("GOVERNMENT = STATE/COUNTY", "GOVERNMENT = FEDERAL/STATE/COUNTY/DISTRICT")
df_new = df_new.replace("GOVERNMENT = COMBINATION GOVT & VOLUNTARY", "UNCLASSIFIED")
df_new

,State,CMS Certification Number (CCN),Provider Name,Address,City,ZIP,Type of Ownership
0,AL,17000,ALABAMA DEPARTMENT OF PUBLIC HEALTH HOME CARE,"201 MONROE STREET, THE RSA TOWER, SUITE 1200",MONTGOMERY,36104,GOVERNMENT - STATE/COUNTY
1,AL,17009,ENCOMPASS HEALTH HOME HEALTH,2970 LORNA ROAD,BIRMINGHAM,35216,PROPRIETARY
2,AL,17013,KINDRED AT HOME,1239 RUCKER BLVD,ENTERPRISE,36330,PROPRIETARY
3,AL,17014,AMEDISYS HOME HEALTH,68278 MAIN STREET,BLOUNTSVILLE,35031,PROPRIETARY
4,AL,17016,SOUTHEAST ALABAMA HOMECARE,804 GLOVER AVENUE,ENTERPRISE,36330,PROPRIETARY
...,...,...,...,...,...,...,...
11171,CA,559124,CARE HOME HEALTH INC,"7200 VINELAND AVENUE, SUITE 213",SUN VALLEY,91352,PROPRIETARY
11172,CA,559125,"BELANI HOME HEALTH CARE, INC","2609 WEST WYOMING AVE, SUITE B",BURBANK,91505,PROPRIETARY
11173,CA,559128,"SAGE HOME HEALTH CARE, INC","20944 SHERMAN WAY, SUITE 108",CANOGA PARK,91303,PROPRIETARY
11174,CA,559132,TEMPLE OF LIFE HOME CARE,"18340 VENTURA BLVD, SUITE 220",TARZANA,91356,PROPRIETARY


In [38]:
df["Type of Ownership"].value_counts()

PROPRIETARY                                     9054
VOLUNTARY NON-PROFIT - PRIVATE                   833
VOLUNTARY NON-PROFIT - OTHER                     584
VOLUNTARY NON PROFIT - RELIGIOUS AFFILIATION     304
GOVERNMENT - STATE/COUNTY                        241
GOVERNMENT - LOCAL                               142
GOVERNMENT - COMBINATION GOVT & VOLUNTARY         18
Name: Type of Ownership, dtype: int64

In [45]:
df_ratings = df[["CMS Certification Number (CCN)", "Provider Name", "Type of Ownership", 
                "Quality of patient care star rating", 
                "How often the home health team began their patients' care in a timely manner", 
                "How often the home health team taught patients (or their family caregivers) about their drugs", 
                "How often the home health team determined whether patients received a flu shot for the current flu season",
                "How often patients got better at walking or moving around",
                "How often patients got better at getting in and out of bed", 
                "How often patients got better at bathing", 
                "How often patients' breathing improved", 
                "How often patients got better at taking their drugs correctly by mouth", 
                "How often home health patients had to be admitted to the hospital", 
                "How often patients receiving home health care needed urgent, unplanned care in the ER without being admitted",
                "How often physician-recommended actions to address medication issues were completely timely", 
                "DTC Risk-Standardized Rate", "DTC Performance Categorization", "PPR Risk-Standardized Rate", 
                "PPR Performance Categorization"]]
df_ratings.head()

,CMS Certification Number (CCN),Provider Name,Type of Ownership,Quality of patient care star rating,How often the home health team began their patients' care in a timely manner,How often the home health team taught patients (or their family caregivers) about their drugs,How often the home health team determined whether patients received a flu shot for the current flu season,How often patients got better at walking or moving around,How often patients got better at getting in and out of bed,How often patients got better at bathing,How often patients' breathing improved,How often patients got better at taking their drugs correctly by mouth,How often home health patients had to be admitted to the hospital,"How often patients receiving home health care needed urgent, unplanned care in the ER without being admitted",How often physician-recommended actions to address medication issues were completely timely,DTC Risk-Standardized Rate,DTC Performance Categorization,PPR Risk-Standardized Rate,PPR Performance Categorization
0,17000,ALABAMA DEPARTMENT OF PUBLIC HEALTH HOME CARE,GOVERNMENT - STATE/COUNTY,4.0,93.4,98.8,55.3,86.3,84.6,86.6,81.4,78.9,14.4,15.8,91.6,88.24,Better Than National Rate,3.34,Same As National Rate
1,17009,ENCOMPASS HEALTH HOME HEALTH,PROPRIETARY,3.5,97.1,99.6,75.4,83.2,80.2,83.4,82.1,79.4,16.0,12.1,98.1,81.10,Better Than National Rate,2.95,Better Than National Rate
2,17013,KINDRED AT HOME,PROPRIETARY,4.0,99.8,99.8,75.0,80.8,79.2,84.2,84.8,73.7,15.4,13.7,91.9,79.59,Same As National Rate,3.62,Same As National Rate
3,17014,AMEDISYS HOME HEALTH,PROPRIETARY,4.5,99.6,99.0,67.3,82.8,82.0,90.2,89.2,83.5,11.0,16.4,96.8,83.12,Better Than National Rate,3.56,Same As National Rate
4,17016,SOUTHEAST ALABAMA HOMECARE,PROPRIETARY,4.5,99.4,100.0,85.7,85.1,87.2,88.0,92.3,84.0,15.7,16.9,96.0,84.63,Better Than National Rate,3.34,Same As National Rate


In [46]:
abbr=[]
hosp_owner=df_ratings["Type of Ownership"] 
length=len(hosp_owner)
for i in range(length):
    if hosp_owner[i] =="VOLUNTARY NON-PROFIT - PRIVATE":
        abbr.append("VNP_P")
    if hosp_owner[i] =="VOLUNTARY NON-PROFIT - OTHER":
        abbr.append("VNP_O")
    if hosp_owner[i] =="VOLUNTARY NON-PROFIT - RELIGIOUS":
        abbr.append("VNP_R")
    if hosp_owner[i] =="GOVERNMENT - FEDERAL/STATE/COUNTY/DISTRICT":
        abbr.append("GOV_FSCD")
    if hosp_owner[i] =="GOVERNMENT - LOCAL":
        abbr.append("GOV_L")
    if hosp_owner[i] =="PROPRIETARY":
        abbr.append("PRO")
    if hosp_owner[i] == "UNCLASSIFIED":
        abbr.append("UNC")

In [47]:
abbr=[]
hosp_owner=df_new["Type of Ownership"] 
length=len(hosp_owner)
for i in range(length):
    if hosp_owner[i] =="VOLUNTARY NON-PROFIT - PRIVATE":
        abbr.append("VNP_P")
    if hosp_owner[i] =="VOLUNTARY NON-PROFIT - OTHER":
        abbr.append("VNP_O")
    if hosp_owner[i] =="VOLUNTARY NON-PROFIT - RELIGIOUS":
        abbr.append("VNP_R")
    if hosp_owner[i] =="GOVERNMENT - FEDERAL/STATE/COUNTY/DISTRICT":
        abbr.append("GOV_FSCD")
    if hosp_owner[i] =="GOVERNMENT - LOCAL":
        abbr.append("GOV_L")
    if hosp_owner[i] =="PROPRIETARY":
        abbr.append("PRO")
    if hosp_owner[i] == "UNCLASSIFIED":
        abbr.append("UNC")

In [48]:
abbr=[]
hosp_owner=df["Type of Ownership"] 
length=len(hosp_owner)
for i in range(length):
    if hosp_owner[i] =="VOLUNTARY NON-PROFIT - PRIVATE":
        abbr.append("VNP_P")
    if hosp_owner[i] =="VOLUNTARY NON-PROFIT - OTHER":
        abbr.append("VNP_O")
    if hosp_owner[i] =="VOLUNTARY NON-PROFIT - RELIGIOUS":
        abbr.append("VNP_R")
    if hosp_owner[i] =="GOVERNMENT - FEDERAL/STATE/COUNTY/DISTRICT":
        abbr.append("GOV_FSCD")
    if hosp_owner[i] =="GOVERNMENT - LOCAL":
        abbr.append("GOV_L")
    if hosp_owner[i] =="PROPRIETARY":
        abbr.append("PRO")
    if hosp_owner[i] == "UNCLASSIFIED":
        abbr.append("UNC")